In [1]:
from selenium import webdriver
import time
import pandas as pd
from tqdm import tqdm
from math import ceil

#Дополнителные
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Работал через Яндекс.Браузер(https://github.com/yandex/YandexDriver). Для этого надо скачать binary file отсюда https://github.com/yandex/YandexDriver/releases

In [2]:
from selenium import webdriver
options = webdriver.ChromeOptions()
binary_yandex_driver_file = 'D:\Downloads\yandexdriver-21.5.0.582-win\yandexdriver.exe' # path to YandexDriver

Спарсим названия и рейтинги фильмов, а также уникальный номер каждого фильма

In [3]:
# Список со всеми названиями фильмов
all_titles = []
# Список со всеми рейтингами фильмов
all_ratings = []
# Список со всеми id-шниками фильмов. Нужен для последующего парсинга отзывов
all_films_ids = []
try:
    browser = webdriver.Chrome(binary_yandex_driver_file, options=options)
    for url in ['https://www.kinopoisk.ru/lists/top250/?page=' + str(i) + '&tab=all' for i in range(1, 6)]:

        browser.get(url) # Открываем страницу

        titles_from_one_page = browser.find_elements_by_css_selector('p.selection-film-item-meta__name')  # browser.find_elements_by_xpath('//*[@class="product_card-title"]/a')
        ratings_from_one_page = browser.find_elements_by_css_selector('span.rating__value.rating__value_positive')#browser.find_elements_by_class_name('rating__value.rating__value_positive')

        # Парсим названия и рейтинги
        titles_from_one_page = [i.text for i in titles_from_one_page]
        all_titles.extend(titles_from_one_page)
        ratings_from_one_page = [i.text for i in ratings_from_one_page]
        all_ratings.extend(ratings_from_one_page)
        # Парсим id фильмов. Id фильма берём из ссылки на страницу фильма
        els = browser.find_elements_by_class_name("selection-film-item-meta [href]")
        links = [el.get_attribute('href') for el in els]
        films_ids_from_one_page = []
        for link_ in links:
            films_ids_from_one_page.append(link_[link_[:-1].rindex('/') + 1:-1])
        all_films_ids.extend(films_ids_from_one_page)   
    
except Exception as e:
    print('Ошибка')
    print(e)
    
finally:

    time.sleep(1)  # Ждем 1 секунду
    browser.quit()  # закрываем браузер после всех манипуляций
    print('Закрыл')

Закрыл


Парсим отзывы на фильмы.

Проблема в том, что на одной странице умещается не больше 200 отзывов. На первой странице с отзывами нахожу, сколько всего отзывов было на фильм, делю на 200 и получаю, сколько страниц с отзывами есть для этого фильма. Потом циклом прохожусь по всем этим страницам.

Почему-то на некоторых страницах с отзывами элемент, отвечающий за число отзывов имеет другой xpath, поэтому его парсил отдельно.

In [6]:
# Список со списками отзывов на фильмы
all_reviews = []
try:
    browser = webdriver.Chrome(binary_yandex_driver_file, options=options)
    
    # Внешниц цикл по всем фильмам
    for id_ in all_films_ids[:125]:
        all_reviews.append([])
        # Парсим отзывы с первой страницы
        browser.get('https://www.kinopoisk.ru/film/' + id_ + '/reviews/ord/date/status/all/perpage/200/')
        reviews = browser.find_elements_by_css_selector('span._reachbanner_')
        reviews = [i.text for i in reviews]
        all_reviews[-1].extend(reviews)

        # Находим на первой странице общее число отзывов
        number_of_reviews_pages = browser.find_elements_by_xpath('//*[@id="block_left_padtop"]/div/table/tbody/tr/td/table/tbody/tr[5]/td/table/tbody/tr/td/div/div[1]/div/div/div[2]')
        if len([i.text for i in number_of_reviews_pages]) > 0:
            number_of_reviews_pages = [i.text for i in number_of_reviews_pages][0]
            number_of_reviews_pages = ceil(int(number_of_reviews_pages[number_of_reviews_pages[:-1].rindex(' ') + 1:]) / 200)
        else:
            number_of_reviews_pages = browser.find_elements_by_xpath('//*[@id="block_left_padtop"]/div/table/tbody/tr/td/table/tbody/tr[5]/td/table/tbody/tr/td/div/div[1]/div/div/div')
            number_of_reviews_pages = [i.text for i in number_of_reviews_pages][0]
            number_of_reviews_pages = ceil(int(number_of_reviews_pages[number_of_reviews_pages[:-1].rindex(' ') + 1:]) / 200)
        # Если отзывов больше 200, то есть страниц с отзывами больше одной, то пробегаемся циклом по всем этим страницам
        # и парсим с них отзывы
        if number_of_reviews_pages > 1:
            for page_number in range(2, number_of_reviews_pages + 1):
                browser.get('https://www.kinopoisk.ru/film/' + id_ + '/reviews/ord/date/status/all/perpage/200/page/' + str(page_number) + '/')
                reviews = browser.find_elements_by_css_selector('span._reachbanner_')
                reviews = [i.text for i in reviews]
                all_reviews[-1].extend(reviews)
except Exception as e:
    print('Ошибка')
    print(e)
    
finally:

    time.sleep(1)  
    browser.quit()  
    print('Закрыл')

Закрыл


При парсинге всех 250 фильмов сразу код почему-то валился, поэтому разбил пополам.

In [36]:
# Список со списками отзывов на фильмы
all_reviews_ = []
try:
    browser = webdriver.Chrome(binary_yandex_driver_file, options=options)
    # Внешниц цикл по всем фильмам
    for id_ in all_films_ids[125:]:
        all_reviews_.append([])
        # Парсим отзывы с первой страницы
        browser.get('https://www.kinopoisk.ru/film/' + id_ + '/reviews/ord/date/status/all/perpage/200/')
        reviews = browser.find_elements_by_css_selector('span._reachbanner_')
        reviews = [i.text for i in reviews]
        all_reviews_[-1].extend(reviews)
        # Находим на первой странице общее число отзывов
        number_of_reviews_pages = browser.find_elements_by_xpath('//*[@id="block_left_padtop"]/div/table/tbody/tr/td/table/tbody/tr[5]/td/table/tbody/tr/td/div/div[1]/div/div/div[2]')
        if len([i.text for i in number_of_reviews_pages]) > 0:
            number_of_reviews_pages = [i.text for i in number_of_reviews_pages][0]
            number_of_reviews_pages = ceil(int(number_of_reviews_pages[number_of_reviews_pages[:-1].rindex(' ') + 1:]) / 200)
        else:
            number_of_reviews_pages = browser.find_elements_by_xpath('//*[@id="block_left_padtop"]/div/table/tbody/tr/td/table/tbody/tr[5]/td/table/tbody/tr/td/div/div[1]/div/div/div')
            number_of_reviews_pages = [i.text for i in number_of_reviews_pages][0]
            number_of_reviews_pages = ceil(int(number_of_reviews_pages[number_of_reviews_pages[:-1].rindex(' ') + 1:]) / 200)
        # Если отзывов больше 200, то есть страниц с отзывами больше одной, то пробегаемся циклом по всем этим страницам
        # и парсим с них отзывы
        if number_of_reviews_pages > 1:
            for page_number in range(2, number_of_reviews_pages + 1):
                browser.get('https://www.kinopoisk.ru/film/' + id_ + '/reviews/ord/date/status/all/perpage/200/page/' + str(page_number) + '/')
                reviews = browser.find_elements_by_css_selector('span._reachbanner_')
                reviews = [i.text for i in reviews]
                all_reviews_[-1].extend(reviews)
except Exception as e:
    print('Ошибка')
    print(e)
    
finally:

    time.sleep(1)  
    browser.quit()  
    print('Закрыл')

Закрыл


Итоговый датафрейм с названиями, рейтингами и отзывами

In [39]:
df = pd.DataFrame({'Название': all_titles, 'Рейтинг': all_ratings, 'Отзывы': all_reviews + all_reviews_})
df

,Название,Рейтинг,Отзывы
0,Побег из Шоушенка,8.9,"[Свобода — одна из самых дорогих вещей в мире,..."
1,Зеленая миля,8.9,"[Не читая книгу, мне трудно судить насколько ф..."
2,Властелин колец: Возвращение короля,8.8,[«Возвращением короля» заканчивается великая т...
3,Интерстеллар,8.8,[Перед нами очередное детище Кристофера Нолана...
4,Властелин колец: Две крепости,8.7,[Довольно тяжело писать рецензию на каждый фил...
...,...,...,...
245,Огни большого города,7.9,"[Не думал, что мне так зайдёт этот фильм. Твор..."
246,"Знакомьтесь, Джо Блэк",7.9,[Первый раз смотрела его кусками - осталась в ...
247,За бортом,7.9,"[«За бортом», или, как он назывался во времена..."
248,Звёздные войны: Эпизод 1 – Скрытая угроза,7.9,"[Сразу хочется начать с того, что вторая (тепе..."
